In [10]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import mean_squared_error
from skimage.measure import compare_ssim as SSIM
import numpy as np
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
_optimizer = Adam(0.0002, 0.5)
from keras import losses

# from tensorflow.keras import layers
# from tensorflow.keras.models import Model
import cv2
import matplotlib.pyplot as plt
from IPython import display
from tqdm import tqdm

In [11]:
def load_imgs(path, number, train_type):
    result=np.empty((number, 72, 72, 3), dtype="float64")
    for i in range(number):
        I = cv2.imread(path + "{:05}_{}.jpeg".format(i+1, train_type))
        result[i, :, :, :] = I
    return result/result.max()

In [12]:
'''load images, parse test/validation set'''
dataNum = 4000
dataPath = "testcase_mass/"
x1 = load_imgs(dataPath, dataNum, 1)
x2 = load_imgs(dataPath, dataNum, 2)
y = load_imgs(dataPath, dataNum, 0)
y = y[:, 8:-8, 8:-8, :]
mask1 = load_imgs(dataPath, dataNum, 4)
mask1 = mask1[:, 8:-8, 8:-8, :1]
mask2 = 1-mask1
mask = np.concatenate((mask1, mask2), axis = 3)

x_train1, x_test1, x_train2, x_test2, y_train, y_test, mask_train, mask_test = train_test_split(
    x1, x2, y, mask, test_size=0.25)

In [13]:
# input size: Nonex48x48x3
# output size: Nonex36x36x16
def pre_convblock(x):
    y = layers.Conv2D(filters = 16, kernel_size = (5, 5), padding = "valid", 
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(x)
    y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters = 32, kernel_size = (5, 5), padding = "valid", 
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(y)
    y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters = 16, kernel_size = (5, 5), padding = "valid", 
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(y)
    y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
    y = layers.ReLU()(y)
    return y

In [14]:
# tensor format: [batch, in_height, in_width, in_channels]
# input size: Nonex36x36x32 (cat: 16+16)
# output size: Nonex32x32x2
# softmax is applied along the channel axis.
def post_convblock(x):
    y = layers.Conv2D(filters = 64, kernel_size=(3, 3), padding = "valid",
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(x)
    y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters = 32, kernel_size=(1, 1), padding = "valid",
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(y)
    y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters = 2, kernel_size=(3, 3), padding = "valid", activation = 'tanh',
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(y)

    y = layers.Softmax(axis = -1)(y)
    return y

In [15]:
# input: out-of-focus image block A & block B, which denote the same area of the whole picture.
#        each: 48x48x3
# output: 
def fusionnet(inTensor1, inTensor2):
    out1 = pre_convblock(inTensor1)
    out2 = pre_convblock(inTensor2)

    x = layers.Concatenate(axis = -1)([out1, out2])
    y = post_convblock(x)
    
    return y

In [16]:
def fusionnetPos(y):
    y, inTensor1, inTensor2 = y
    # crop the input images to the same size as network output.
    inCrop1 = layers.Cropping2D(cropping=((8, 8), (8, 8)))(inTensor1)
    inCrop2 = layers.Cropping2D(cropping=((8, 8), (8, 8)))(inTensor2)
    y1 = y[:, :, :, :1]
    y2 = y[:, :, :, 1:]
    y1 = K.tile(y1, [1, 1, 1, 3])     # extend y1&y2 dimension to 3, consistant to color channels
    y2 = K.tile(y2, [1, 1, 1, 3])
    y1 = layers.Multiply()([inCrop1, y1])
    y2 = layers.Multiply()([inCrop2, y2])
    y = layers.Add()([y1, y2])
    return y

In [17]:
img1 = layers.Input(shape=(72, 72, 3))
img2 = layers.Input(shape=(72, 72, 3))
intermed = fusionnet(img1, img2) # intermed: mask layer
pred = layers.Lambda(fusionnetPos)([intermed, img1, img2])

In [18]:
'''2 outputs: 'pred' for GAN loss and 'intermed' for mask loss'''
generator = Model(inputs = [img1, img2], outputs = [pred, intermed])
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 68, 68, 16)   1216        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 68, 68, 16)   1216        input_4[0][0]                    
__________________________________________________________________________________________________
batch_norm

In [19]:
'''tv_loss: designed constraint on mask. WORKS NOT WELL.'''

def tv_loss(y_true, y_pred):
    #mapping = tf.cast(y_pred > 0.5, y_pred.dtype)
    #loss = tf.reduce_mean(tf.image.total_variation(mapping))

#     mapping = 0.25 - tf.square(y_pred-0.5) # 0.25-(x-0.5)^2
#     sigma = 0.2
#     mu = 0.5
#     mapping = tf.exp(-0.5*tf.square((y_pred - mu)/sigma))
    mapping = tf.square(y_pred)*tf.square(y_pred-1)/(tf.square(y_pred-0.5)+0.25)
    loss = tf.reduce_mean(mapping)
    return loss

In [20]:
# '''train on generator using MSE of pred & mask.'''

# lambda_tv = 1
# generator.compile(loss=[losses.mean_squared_error, tv_loss], loss_weights=[1, 0.001], optimizer= _optimizer)
# generator.fit([x_train1, x_train2], [y_train, mask_train], batch_size=64, epochs=5)



# prediction, a = generator.predict([x1, x2])
# get_layer_output = K.function([generator.layers[0].input, generator.layers[1].input],
#                                   [generator.layers[-2].output])

# layer_output = np.array(get_layer_output([x1, x2])[0])

# imgIdx = 6
# fig=plt.figure(figsize=(12, 12))
# columns = 6
# rows = 1
# fig.add_subplot(rows, columns, 1)
# plt.imshow(x1[imgIdx, 8:-8, 8:-8, :])
# fig.add_subplot(rows, columns, 2)
# plt.imshow(x2[imgIdx, 8:-8, 8:-8, :])
# fig.add_subplot(rows, columns, 3)
# plt.imshow(mask[imgIdx, :, :, 0])
# fig.add_subplot(rows, columns, 4)
# plt.imshow(layer_output[imgIdx, :, :, 0])
# fig.add_subplot(rows, columns, 5)
# plt.imshow(prediction[imgIdx, :, :, :])
# fig.add_subplot(rows, columns, 6)
# plt.imshow(y[imgIdx, :, :, :])
# plt.show()
# # fig.savefig(dataPath+"results/struc_loss{:.2E}.png".format(lambda_tv))

In [21]:
def dis_block(x, _filters, _strides, bn = True):
    y = layers.Conv2D(filters = _filters, kernel_size = (5, 5), strides = _strides, padding='same',
                     kernel_initializer = keras.initializers.RandomNormal(stddev=0.02))(x)
    if bn:
        y = layers.BatchNormalization(momentum = 0.9, epsilon=1e-5)(y)
        
    y = layers.LeakyReLU(alpha=0.2)(y)
    return y

In [22]:
def disnet(x):
    y = dis_block(x, 32, (2, 2), bn = False)
    y = dis_block(y, 64, (1, 1))
    y = dis_block(y, 64, (2, 2))
    y = dis_block(y, 128, (1, 1))
    y = dis_block(y, 128, (2, 2))
#     y = dis_block(y, 256, (2, 2))
#     y = dis_block(y, 256, (2, 2))
    y = layers.Flatten()(y)
    y = layers.Dense(1, activation='sigmoid')(y)
    return y

In [23]:
dis_input = layers.Input(shape=(56, 56, 3))
dis_output = disnet(dis_input)
discriminator = Model(inputs = dis_input, outputs = dis_output)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 56, 56, 3)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        2432      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 64)        51264     
_________________________________________________________________
batch_normalization_17 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 64)        102464    
__________

In [24]:
# discriminator.compile(loss='mse', optimizer= _optimizer)
# y_fake = generator.predict([x_train1, x_train2])
# dis_input = np.concatenate((y_fake, y_train), axis = 0)
# label = np.append(np.zeros((y_fake.shape[0], )), np.ones((y_train.shape[0], )))

# for e in range(20):
#     discriminator.fit(dis_input, label)

# label = discriminator.predict(x_test1[:, 8:-8, 8:-8, :])
# print(np.sum(label < 0.5))
# print(label)

In [25]:
def gannet(x):
    img1, img2 = x
    pred, intermed = generator([img1, img2])
    prob = discriminator(pred)
    discriminator.trainable = False
    return (prob, intermed)

In [26]:
prob, intermed = gannet([img1, img2])
gan = Model(inputs = [img1, img2], outputs= [prob, intermed])
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 72, 72, 3)    0                                            
__________________________________________________________________________________________________
model_2 (Model)                 [(None, 56, 56, 3),  75778       input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 1)            778625      model_2[1][0]                    
Total para

In [28]:
# prediction = gan.predict([x1, x2])
# mask = prediction[1]
# plt.imshow(mask[6, :, :, 0])
# mask_tensor = tf.convert_to_tensor(mask)
# tv = tf.image.total_variation(mask_tensor)
# sess = tf.Session()
# loss = tv.eval(session=sess)

# print(loss[:10]/32/32/2)

In [29]:
def plot_loss(losses):
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plt.figure(figsize=(10,8))
    plt.plot(losses["d"], label='discriminitive loss')
    plt.plot(losses["g"], label='generative loss')
    plt.legend()
    plt.show()

In [30]:
epoch = 100
batchSize = 64
losses = {"d":[], "g":[]}
for e in tqdm(range(epoch)):
    rand_idx = np.random.randint(0, x_train1.shape[0], size = batchSize)
    img_batch1 = x_train1[rand_idx, :, :, :]
    img_batch2 = x_train2[rand_idx, :, :, :]
    mask_batch = mask_train[rand_idx, :, :, :]
    y_batch = y_train[np.random.randint(0, y_train.shape[0], size = batchSize), :, :, :]
    img_fake = generator.predict([img_batch1, img_batch2])[0]
    img_valid = y_batch
    dis_input = np.concatenate((img_fake, img_valid), axis = 0)
    label = np.append(np.zeros((batchSize, )), np.ones((batchSize, )))
    
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=_optimizer)
    d_loss = discriminator.train_on_batch(dis_input, label)
    losses["d"].append(d_loss)
    
    gan_label = np.ones((batchSize,))
    discriminator.trainable = False
    gan.compile(loss = ['binary_crossentropy', tv_loss], loss_weights=[1, 0], optimizer=_optimizer)

    g_loss = gan.train_on_batch([img_batch1, img_batch2], [gan_label, mask_batch])
    losses["g"].append(g_loss[1])
    
    if e % 5 == 4:
        plot_loss(losses)

        imgIdx = 16
        prediction = generator.predict([x1[imgIdx:imgIdx+1, :, :, :], x2[imgIdx:imgIdx+1, :, :, :]])[0]
        #ssim = SSIM(y[imgIdx, :, :, :], prediction[0, :, :, :], data_range=prediction[0, :, :, :].max() - prediction[0, :, :, :].min(), multichannel=True)
        #print(ssim)
        
        get_layer_output = K.function([generator.layers[0].input, generator.layers[1].input],
                                          [generator.layers[-2].output])

        layer_output = np.array(get_layer_output([x1[imgIdx:imgIdx+1, :, :, :], x2[imgIdx:imgIdx+1, :, :, :]])[0])

        fig=plt.figure(figsize=(12, 12))
        columns = 6
        rows = 1
        fig.add_subplot(rows, columns, 1)
        plt.imshow(x1[imgIdx, 8:-8, 8:-8, :])
        fig.add_subplot(rows, columns, 2)
        plt.imshow(x2[imgIdx, 8:-8, 8:-8, :])
        fig.add_subplot(rows, columns, 3)
        plt.imshow(mask[imgIdx, :, :, 0])
        fig.add_subplot(rows, columns, 4)
        plt.imshow(layer_output[0, :, :, 0])
        fig.add_subplot(rows, columns, 5)
        plt.imshow(prediction[0, :, :, :])
        fig.add_subplot(rows, columns, 6)
        plt.imshow(y[imgIdx, :, :, :])
        plt.show()
        #fig.savefig(dataPath+"results/struc_ep{}_{}.png".format(e,98))

  0%|          | 0/100 [00:00<?, ?it/s]


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_10/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_10/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_10/kernel/read)]]
	 [[{{node lambda_2/add_3/add/_539}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_405_lambda_2/add_3/add", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
# prediction = generator.predict([x_test1, x_test2])
# get_layer_output = K.function([generator.layers[0].input, generator.layers[1].input],
#                                   [generator.layers[-2].output])

# layer_output = np.array(get_layer_output([x_test1, x_test2])[0])

In [ ]:
# np.save('x_test1.npy', x_test1)
# np.save('x_test2.npy', x_test2)
# np.save('y_test.npy', y_test)
# np.save('prediction.npy', prediction[0])
# np.save('layer_output.npy', layer_output)
# np.save('mask_test.npy', mask_test)

In [ ]:
# # ssimList = []
# # ada = 999
# for imgIdx in range(1000):
# #     ssim = SSIM(y_test[imgIdx, :, :, :], prediction[0][imgIdx, :, :, :], data_range=1, multichannel=True)
# #     ssimList.append(ssim)
# #     ssim1 = SSIM(y_test[imgIdx, :, :, :], x_test1[imgIdx, 8:-8, 8:-8, :], data_range=1, multichannel=True)
# #     ssim2 = SSIM(y_test[imgIdx, :, :, :], x_test2[imgIdx, 8:-8, 8:-8, :], data_range=1, multichannel=True)
# #     print(ssim1)
# #     print(ssim2)
# #     print(ssim)
# #    imgIdx = 312
#     fig=plt.figure(figsize=(12, 12))
#     columns = 6
#     rows = 1
#     fig.add_subplot(rows, columns, 1)
#     plt.imshow(x_test1[imgIdx, 8:-8, 8:-8, :])
#     fig.add_subplot(rows, columns, 2)
#     plt.imshow(x_test2[imgIdx, 8:-8, 8:-8, :])
#     fig.add_subplot(rows, columns, 3)
#     plt.imshow(layer_output[imgIdx, :, :, 0])
#     fig.add_subplot(rows, columns, 4)
#     plt.imshow(prediction[0][imgIdx, :, :, :])
#     fig.add_subplot(rows, columns, 5)
#     plt.imshow(y_test[imgIdx, :, :, :])
#     fig.add_subplot(rows, columns, 6)
#     plt.imshow(mask_test[imgIdx, :, :, 0])
#     plt.show()
#     fig.savefig(dataPath+"results/{}.png".format(imgIdx))

In [ ]:
# plt.hist(ssimList, bins = 20)

In [ ]:
# generator.save('generator.h5')